In [ ]:
import cv2
import numpy as np
import urllib.request
import os
from PIL import Image
import matplotlib.pyplot as plt

# 画像をダウンロードする関数
def download_image(url, filename):
    urllib.request.urlretrieve(url, filename)
    return filename

# 画像の類似度を計算する関数
def calculate_structural_similarity(image1_path, image2_path):
    # 画像をグレースケールで読み込み
    img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)
    
    # AKAZE特徴量抽出器を作成
    akaze = cv2.AKAZE_create()

    # 各画像から特徴点と記述子を検出
    keypoints1, descriptors1 = akaze.detectAndCompute(img1, None)
    keypoints2, descriptors2 = akaze.detectAndCompute(img2, None)

    # ディスクリプタのタイプを確認し、必要に応じて変換
    if descriptors1.dtype != np.float32:
        descriptors1 = descriptors1.astype(np.float32)
    if descriptors2.dtype != np.float32:
        descriptors2 = descriptors2.astype(np.float32)

    # FLANNベースのマッチャーを作成
    # マッチャーのパラメータ設定
    index_params = dict(algorithm=1, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # マッチングを実行
    matches = flann.match(descriptors1, descriptors2)

    # マッチングの距離でソート（低いほど良いマッチ）
    matches = sorted(matches, key=lambda x: x.distance)

    # 類似度スコアを計算（使用されたマッチングの数と距離に基づく）
    score = sum([m.distance for m in matches]) / len(matches)
    return score

image1_path = "../sample_data/coco_sample_datasets/sample_coco_train2017/000000187976.jpg"
image2_path = "../sample_data/coco_sample_datasets/sample_coco_train2017/000000187976.jpg"



# 類似度スコアを計算
similarity_score = calculate_structural_similarity(image1_path, image2_path)


# 画像をPIL形式に変換
img1_pil = Image.open(image1_path)
img2_pil = Image.open(image2_path)

# 画像を表示
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(img1_pil, cmap='gray')
axes[0].set_title('Image 1')
axes[0].axis('off')
axes[1].imshow(img2_pil, cmap='gray')
axes[1].set_title('Image 2')
axes[1].axis('off')
plt.show()

similarity_score
